In [80]:
from typing import Union, List, Dict, Tuple, Literal
from crimson.py_ts_types_beta.convert_typing.positive import SubscribeNode
from crimson.py_ts_types_beta.convert_typing.negative import convert_final
from crimson.py_ts_types_beta.convert_typing import convert_py_to_ts, convert_ts_to_py
import inspect


## py - ts mapping

The python types are converted as below:

``` ts

export type str = string
export type int = number
export type float = number
export type bool = boolean
export type None = null | undefined

export type List<T> = Array<T>
export type Dict<K extends keyof any = string, V = any> = {
    [key in K extends string | number ? K : string]: V;
}
export type Tuple<T extends any[]> = T
export type Union<T> = T
export type Literal<T extends string | number | boolean> = T
export type Optional<T> = T | undefined
export type Any = any
export type Callable<T extends any[], R> = (...args: T) => R

```


## Convert Positive

First, we will convert a python type into the typescrit type.

In the example type, I added some strings including `>`, `[`, `]`.

They are important brackets in python and typescript syntaxes.

They won't cause any problem.


In [81]:
Union[int,List[int],Union[Tuple[Dict[str,Dict[str,int]],Literal['[bye>',2,'[hi]']],int]]

typing.Union[int, typing.List[int], typing.Tuple[typing.Dict[str, typing.Dict[str, int]], typing.Literal['[bye>', 2, '[hi]']]]

For the purpose, we use `anytree` node.

Construct a root node, and use `generation_children` and `to_string`,

and we get the converted type.

In [82]:
py_annotation = """Union[int,List[int],Union[Tuple[Dict[str,Dict[str,int]],Literal['[bye>',2,'[hi]']],int]]"""

root = SubscribeNode(converted=[py_annotation])
root.generate_children()

ts_annotation = root.to_string()
ts_annotation

"Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<'[bye>'|2|'[hi]'>]>|int>>"

``` ts
Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<'[bye>'|2|'[hi]'>]>|int>>"
```

We can use a shortcut called 'convert_py_to_ts' as well.


In [83]:
convert_py_to_ts(py_annotation)

"Union<int|List<int>|Union<Tuple<[Dict<str,Dict<str,int>>,Literal<'[bye>'|2|'[hi]'>]>|int>>"

## Convert Negative

I don't know a reliable typescript parser `python` library.

Therefore, I implemented the negative conversion without a parser.

The `convert_final` function integrates helper functions to convert the typescript type into python type back.

In [84]:
print(inspect.getsource(convert_final))

def convert_final(annotation_string: str) -> str:

    string_free_string, extracted_strings, used_quotes = tokenize_and_extract_strings(
        input_string=annotation_string,
        quotes=["'", "'''", '"', '"""'],
        string_token="text",
    )

    string_free_string = string_free_string.replace(" ", "")

    converted = convert(tokenized_annotation_string=string_free_string)

    for i, extracted_string in enumerate(extracted_strings):
        token = f"text{i}"
        quote = used_quotes[i]
        converted = converted.replace(token, f"{quote}{extracted_string}{quote}")

    return converted



`convert_py_to_ts` is just a renamed `convert_final`.

You can use one of them.

In [85]:
convert_ts_to_py(ts_annotation)

"Union[int,List[int],Union[Tuple[Dict[str,Dict[str,int]],Literal['[bye>',2,'[hi]']],int]]"

In [86]:
py_annotation_recovered = convert_final(ts_annotation)
py_annotation_recovered

"Union[int,List[int],Union[Tuple[Dict[str,Dict[str,int]],Literal['[bye>',2,'[hi]']],int]]"

Check if the original py_annotation was recovered.

In [87]:
py_annotation == py_annotation_recovered

True